In [54]:
import os
import re
import numpy as np
import pandas as pd
from datasets import load_dataset, Dataset, ClassLabel
from langchain.document_loaders import PyPDFLoader
from utils import read_json
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics.pairwise import cosine_similarity

In [27]:
def get_whole_pdf(file_data):
    whole_pdf = ""
    for i in range(len(file_data)):
        whole_pdf += file_data[i].page_content

    return whole_pdf

In [28]:
configs = read_json('configs.json')
DATA_PATH = configs["DATA_PATH"]
DB_PATH = configs['DB_PATH']
files_list = [file for file in os.listdir(DATA_PATH) if os.path.isfile(os.path.join(DATA_PATH, file))]

def read_pdf(files_list, data_path=DATA_PATH):
    content_dict = {}

    for file_name in files_list:
        file_path = data_path + file_name
        loader = PyPDFLoader(file_path)
        content = loader.load()
        whole_pdf = ""
        for i in range(len(content)):
            whole_pdf += content[i].page_content

        whole_pdf = re.sub(r'[：。\n]', '', whole_pdf)
        content_dict[file_name] = whole_pdf

    return content_dict

In [29]:
files_list

['34944_高鐵_促參.pdf',
 '38005_核四_政策.pdf',
 '39243_核四_品質.pdf',
 '39477_核四_料件.pdf',
 '46365_高鐵_基金.pdf',
 '48052_大客車_逃生門.pdf',
 '48746_大客車_安全門.pdf',
 '49490_核四_延宕.pdf',
 '49627_核四_停建.pdf',
 '49676_大客車_駕照.pdf',
 '52922_大客車_超時.pdf',
 '54210_高鐵_出資.pdf',
 '54376_高鐵_航發.pdf',
 '54561_高鐵_機電.pdf',
 '58930_核四_封存.pdf']

In [30]:
pdf_contents = read_pdf(files_list)

In [31]:
def content_extraction(input, pattern):

    # Search using the pattern
    match = re.search(pattern, input, re.S)

    if match:
        result = match.group(1).strip()  # Use strip() to remove any leading/trailing whitespace
        print("Extraction succeed.")
        return result
    else:
        print("No match found")
        pass

In [32]:
pattern = r"貳、案   由(.*?)參、事實與理由"

extracted_content = {}
for filename, content in pdf_contents.items():
    extracted_content[filename] = content_extraction(content, pattern)

Extraction succeed.
Extraction succeed.
Extraction succeed.
Extraction succeed.
Extraction succeed.
Extraction succeed.
Extraction succeed.
Extraction succeed.
Extraction succeed.
Extraction succeed.
Extraction succeed.
Extraction succeed.
Extraction succeed.
Extraction succeed.
Extraction succeed.


In [33]:
def extract_label(filename):
    # Assuming the label is always after the first underscore and ends before the second underscore
    match = re.search(r'_(.*?)_', filename)
    if match:
        return match.group(1)  # This extracts text between the first pair of underscores
    return None  # In case the pattern does not match

In [34]:
formatted_data  = [{'label': extract_label(filename), 'text' : content } for filename, content in extracted_content.items()]
formatted_data 

[{'label': '高鐵',
  'text': '交通部於 87年間與臺灣高速鐵路公司簽訂「臺灣南北高速鐵路興建營運合約」及「臺灣南北高速鐵路站區開發合約 」，疏未預先審度該公司展延高鐵通車營運時程及遲延受領站區用地所生 損失之責任歸屬，並綢繆約定 相關處罰條款 ，嗣對政府權益保障恝置不察，一再同意該公司展延通車營運時程及受領站區用地， 致政府蒙受回饋金與租金收入減少及顧問費用增加等鉅額損失，均有違失，爰依監察法第 24條規定提案糾正'},
 {'label': '核四',
  'text': '核四封存後每年仍耗費數億元於資產維護管理，行政院及經濟部對外宣告核四重啟不可行，核四興建費用 2,833億元頇列為損失，行政院及 經濟部對核四政策之重大變動，導致資源嚴重浪費； 再者，經濟部宣布能源配比 (燃氣50%、燃煤30%、再生能源20%)之能源轉型政策，未經能源安全、能源經濟及環境影響等完整評估 ，復於再生能源發電量增加有限情況下， 以運轉中核電機組長期停機 方式減核 ，致近年火力發購電量逐年提高， 106年占比達84.4%，燃煤發電增幅 甚至高於燃氣，造成嚴重空氣污染；以及經濟部宣布 新能源政策 之前，並未評估其對電價之影響，迄 106年3月行政院始於國公營企業體檢小組會議評估 等情，均有違失，爰依法提案糾正'},
 {'label': '核四',
  'text': '台電公司 未落實「核四工程品質保證方案」，致龍門電廠試運轉時違規與注意改善事項層出不窮，如 抑壓池灌水作業 不當，致反應器廠房底層淹水 、壓力試驗合格之室內消防栓 系統，其 太平龍頭 竟脫落，致汽機廠房積水等 ，均嚴重衝擊國人對核能安全運轉之信心等情 ，確有諸多違失，爰依法提案糾正'},
 {'label': '核四',
  'text': '台灣電力股份有限公司 (下稱台電公司 )於本院調查核四廠一號機因施工測詴期間設備損壞而移用二號機相關設備之過程，所提供資料內容前後不一，設備組件損壞、採購及修復個數未能確實清查正確，顯見台電公司核四廠之料件管理系統紊亂，且回復本院公文一再發生資料正確性不足，核有怠失；另台電公司於 81年陳報核四興建計畫， 竟以69年所估算成本陳報， 未能如實報告核四建廠成本，致使政府無法確實評估該項投資計畫之成本效益，台電公司表示當時即考慮日後再以追加預算方式提出，此

In [35]:
label = []
for i in range(len(formatted_data)):
    current_label = formatted_data[i]['label']
    if current_label not in label:
        label.append(current_label)
    else:
        continue
label_feature = ClassLabel(names=label)
label_feature

ClassLabel(names=['高鐵', '核四', '大客車'], id=None)

In [36]:
dataset = Dataset.from_dict({
    'text': [item['text'] for item in formatted_data],
    'label': [item['label'] for item in formatted_data]
})
dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 15
})

In [37]:
def preprocess_label(example):
    
    example['label'] = [label_feature.str2int(label) for label in example['label']]
    return example

In [38]:
dataset_processed = dataset.map(preprocess_label, batched=True)
dataset_split = dataset_processed.train_test_split(test_size=0.3)

Map:   0%|          | 0/15 [00:00<?, ? examples/s]

In [39]:
dataset_split

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 10
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 5
    })
})

## distilbert-base-cased

In [40]:
model_ckpt = "distilbert-base-cased"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = AutoModel.from_pretrained(model_ckpt).to(device)
tokenizer = DistilBertTokenizer.from_pretrained(model_ckpt)
model = DistilBertForSequenceClassification.from_pretrained(model_ckpt, num_labels=3)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [41]:
def extract_hidden_states(batch):
    # Place model inputs on the GPU
    inputs = {k:v.to(device) for k,v in batch.items() 
              if k in tokenizer.model_input_names}
    # Extract last hidden states
    with torch.no_grad():
        last_hidden_state = model(**inputs).last_hidden_state
    # Return vector for [CLS] token
    return {"hidden_state": last_hidden_state[:,0].cpu().numpy()}

In [42]:
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

In [43]:
dataset_split

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 10
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 5
    })
})

In [44]:
dataset_encoded = dataset_split.map(tokenize, batched=True, batch_size=None)

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [45]:
dataset_encoded.set_format("torch", 
                            columns=["input_ids",  "label","attention_mask"])

In [46]:
dataset_encoded

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 10
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 5
    })
})

In [47]:
def compute_metrics(pred):
    label = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(label, preds, average="weighted")
    acc = accuracy_score(label, preds)
    return {"accuracy": acc, "f1": f1}

In [48]:
batch_size = 1
logging_steps = len(dataset_encoded["train"]) // batch_size
model_name = f"{model_ckpt}-finetuned-emotion"
training_args = TrainingArguments(output_dir=model_name,
                                  num_train_epochs=20,
                                  learning_rate=2e-5,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  weight_decay=0.01,
                                  evaluation_strategy="epoch",
                                  disable_tqdm=False,
                                  logging_steps=logging_steps,
                                  push_to_hub=False, 
                                  log_level="error")

trainer = Trainer(model=model, args=training_args, 
                  compute_metrics=compute_metrics,
                  train_dataset=dataset_encoded["train"],
                  eval_dataset=dataset_encoded["test"],
                  tokenizer=tokenizer)

# Now you can train the model
trainer.train()


  0%|          | 0/200 [00:00<?, ?it/s]

{'loss': 1.193, 'grad_norm': 7.327645301818848, 'learning_rate': 1.9e-05, 'epoch': 1.0}


  0%|          | 0/5 [00:00<?, ?it/s]

{'eval_loss': 1.1221609115600586, 'eval_accuracy': 0.2, 'eval_f1': 0.06666666666666667, 'eval_runtime': 1.2871, 'eval_samples_per_second': 3.885, 'eval_steps_per_second': 3.885, 'epoch': 1.0}
{'loss': 1.1163, 'grad_norm': 6.4350385665893555, 'learning_rate': 1.8e-05, 'epoch': 2.0}


  0%|          | 0/5 [00:00<?, ?it/s]

...

  0%|          | 0/5 [00:00<?, ?it/s]

{'eval_loss': 1.8593422174453735, 'eval_accuracy': 0.4, 'eval_f1': 0.3333333333333333, 'eval_runtime': 1.5083, 'eval_samples_per_second': 3.315, 'eval_steps_per_second': 3.315, 'epoch': 19.0}
{'loss': 0.154, 'grad_norm': 4.96265983581543, 'learning_rate': 0.0, 'epoch': 20.0}


  0%|          | 0/5 [00:00<?, ?it/s]

{'eval_loss': 1.8191888332366943, 'eval_accuracy': 0.4, 'eval_f1': 0.3333333333333333, 'eval_runtime': 1.522, 'eval_samples_per_second': 3.285, 'eval_steps_per_second': 3.285, 'epoch': 20.0}
{'train_runtime': 203.074, 'train_samples_per_second': 0.985, 'train_steps_per_second': 0.985, 'train_loss': 0.6154014837741851, 'epoch': 20.0}


TrainOutput(global_step=200, training_loss=0.6154014837741851, metrics={'train_runtime': 203.074, 'train_samples_per_second': 0.985, 'train_steps_per_second': 0.985, 'total_flos': 17593640136000.0, 'train_loss': 0.6154014837741851, 'epoch': 20.0})

In [50]:
trainer.save_model(f"./models/{model_ckpt}_finetune")
tokenizer.save_pretrained(f"./models/{model_ckpt}_finetune")

('./models/distilbert-base-cased_finetune\\tokenizer_config.json',
 './models/distilbert-base-cased_finetune\\special_tokens_map.json',
 './models/distilbert-base-cased_finetune\\vocab.txt',
 './models/distilbert-base-cased_finetune\\added_tokens.json')

In [51]:
# Load the model and tokenizer for classification
model_name = f"./models/{model_ckpt}_finetune"
model = DistilBertForSequenceClassification.from_pretrained(model_name)
tokenizer = DistilBertTokenizer.from_pretrained(model_name)
model.eval()


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [52]:
def get_embeddings(texts, model, tokenizer):
    # Tokenize the input texts
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True)
    # Pass inputs through the model to get hidden states
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)
    # Get the embeddings from the last hidden state
    hidden_states = outputs.hidden_states[-1]
    # Average the hidden states to get a single vector representation for each text
    embeddings = hidden_states.mean(dim=1)
    return embeddings


In [55]:
# Extract texts from the dictionary
texts = list(extracted_content.values())

# Get embeddings for all documents
embeddings = get_embeddings(texts, model, tokenizer)

# Compute cosine similarity between all pairs of embeddings
cosine_similarities = cosine_similarity(embeddings)

# Convert cosine similarity matrix to a DataFrame
doc_keys = list(extracted_content.keys())
cosine_sim_df = pd.DataFrame(cosine_similarities, index=doc_keys, columns=doc_keys)


In [56]:
cosine_sim_df

,34944_高鐵_促參.pdf,38005_核四_政策.pdf,39243_核四_品質.pdf,39477_核四_料件.pdf,46365_高鐵_基金.pdf,48052_大客車_逃生門.pdf,48746_大客車_安全門.pdf,49490_核四_延宕.pdf,49627_核四_停建.pdf,49676_大客車_駕照.pdf,52922_大客車_超時.pdf,54210_高鐵_出資.pdf,54376_高鐵_航發.pdf,54561_高鐵_機電.pdf,58930_核四_封存.pdf
34944_高鐵_促參.pdf,1.000000,0.307448,0.968530,0.377248,0.943111,0.636608,0.498878,0.295658,0.343623,0.565957,0.551813,0.475684,0.521782,0.921647,0.488265
38005_核四_政策.pdf,0.307448,1.000000,0.301577,0.969869,0.196281,0.257655,0.503576,0.979706,0.986909,0.468108,0.554060,0.793711,0.882518,0.111021,0.922554
39243_核四_品質.pdf,0.968530,0.301577,1.000000,0.374225,0.951338,0.693962,0.513200,0.276348,0.336366,0.591662,0.581974,0.468571,0.526780,0.946012,0.497873
39477_核四_料件.pdf,0.377248,0.969869,0.374225,1.000000,0.259495,0.377033,0.607072,0.956238,0.981471,0.576461,0.666043,0.855767,0.931690,0.178574,0.951086
46365_高鐵_基金.pdf,0.943111,0.196281,0.951338,0.259495,1.000000,0.591172,0.389930,0.184293,0.227988,0.461557,0.445688,0.351613,0.410240,0.971414,0.384899
48052_大客車_逃生門.pdf,0.636608,0.257655,0.693962,0.377033,0.591172,1.000000,0.834267,0.204211,0.292695,0.887424,0.860195,0.550485,0.551045,0.614821,0.429632
48746_大客車_安全門.pdf,0.498878,0.503576,0.513200,0.607072,0.389930,0.834267,1.000000,0.461741,0.539070,0.969393,0.948794,0.812539,0.740494,0.367981,0.609379
49490_核四_延宕.pdf,0.295658,0.979706,0.276348,0.956238,0.184293,0.204211,0.461741,1.000000,0.981976,0.421180,0.505510,0.766209,0.853590,0.092226,0.906834
49627_核四_停建.pdf,0.343623,0.986909,0.336366,0.981471,0.227988,0.292695,0.539070,0.981976,1.000000,0.502697,0.587410,0.822301,0.911249,0.141063,0.948334
49676_大客車_駕照.pdf,0.565957,0.468108,0.591662,0.576461,0.461557,0.887424,0.969393,0.421180,0.502697,1.000000,0.961105,0.753468,0.719094,0.455117,0.593920


In [58]:
cosine_sim_df['58930_核四_封存.pdf'].sort_values(ascending=False)

58930_核四_封存.pdf      1.000000
54376_高鐵_航發.pdf      0.963519
39477_核四_料件.pdf      0.951086
49627_核四_停建.pdf      0.948334
38005_核四_政策.pdf      0.922554
49490_核四_延宕.pdf      0.906834
54210_高鐵_出資.pdf      0.843521
52922_大客車_超時.pdf     0.677468
48746_大客車_安全門.pdf    0.609379
49676_大客車_駕照.pdf     0.593920
39243_核四_品質.pdf      0.497873
34944_高鐵_促參.pdf      0.488265
48052_大客車_逃生門.pdf    0.429632
46365_高鐵_基金.pdf      0.384899
54561_高鐵_機電.pdf      0.303653
Name: 58930_核四_封存.pdf, dtype: float32

In [57]:
cosine_sim_df['52922_大客車_超時.pdf'].sort_values(ascending=False)

52922_大客車_超時.pdf     1.000000
49676_大客車_駕照.pdf     0.961105
48746_大客車_安全門.pdf    0.948794
48052_大客車_逃生門.pdf    0.860195
54210_高鐵_出資.pdf      0.831941
54376_高鐵_航發.pdf      0.797276
58930_核四_封存.pdf      0.677468
39477_核四_料件.pdf      0.666043
49627_核四_停建.pdf      0.587410
39243_核四_品質.pdf      0.581974
38005_核四_政策.pdf      0.554060
34944_高鐵_促參.pdf      0.551813
49490_核四_延宕.pdf      0.505510
46365_高鐵_基金.pdf      0.445688
54561_高鐵_機電.pdf      0.424013
Name: 52922_大客車_超時.pdf, dtype: float32

In [59]:
cosine_sim_df['54210_高鐵_出資.pdf'].sort_values(ascending=False)

54210_高鐵_出資.pdf      1.000000
54376_高鐵_航發.pdf      0.913352
39477_核四_料件.pdf      0.855767
58930_核四_封存.pdf      0.843521
52922_大客車_超時.pdf     0.831941
49627_核四_停建.pdf      0.822301
48746_大客車_安全門.pdf    0.812539
38005_核四_政策.pdf      0.793711
49490_核四_延宕.pdf      0.766209
49676_大客車_駕照.pdf     0.753468
48052_大客車_逃生門.pdf    0.550485
34944_高鐵_促參.pdf      0.475684
39243_核四_品質.pdf      0.468571
46365_高鐵_基金.pdf      0.351613
54561_高鐵_機電.pdf      0.278485
Name: 54210_高鐵_出資.pdf, dtype: float32

In [60]:
cosine_sim_df['54376_高鐵_航發.pdf'].sort_values(ascending=False)

54376_高鐵_航發.pdf      1.000000
58930_核四_封存.pdf      0.963519
39477_核四_料件.pdf      0.931690
54210_高鐵_出資.pdf      0.913352
49627_核四_停建.pdf      0.911249
38005_核四_政策.pdf      0.882518
49490_核四_延宕.pdf      0.853590
52922_大客車_超時.pdf     0.797276
48746_大客車_安全門.pdf    0.740494
49676_大客車_駕照.pdf     0.719094
48052_大客車_逃生門.pdf    0.551045
39243_核四_品質.pdf      0.526780
34944_高鐵_促參.pdf      0.521782
46365_高鐵_基金.pdf      0.410240
54561_高鐵_機電.pdf      0.329598
Name: 54376_高鐵_航發.pdf, dtype: float32